In [ ]:
#Imports
import torch
import numpy as np 
import time # For meassuring running time of the training.

## For getting the local py files
# For Google Drive
#import sys
#sys.path.append('/content/drive/MyDrive/Master/Semester III/Project/')

from ResNET import Block, ResNET
from helper_functions import to_categorical, calculate_accuracy, prepare_data_for_resnet

# Disabling all the warnings in the final version
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
path = './data/**/*.png'
data_size = 10
batch_size = 2
train_loader, valid_loader, test_loader = prepare_data_for_resnet(path, data_size, batch_size)

### Model Implementation

#### Configurations

In [ ]:
#Configurations

num_epochs = 50
lr = 0.001
batch_size = 128
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_classes = 2
img_channel = 3

__Assert block__ to check if the implemented Model class return the correct label dim

In [ ]:
def test():
    resnet = ResNET(Block, [3, 4, 6, 3], img_channel, num_classes)
    _x = torch.randn(2, 3, 244, 244)
    _y = resnet(_x).to(device)
    return (_y.shape)
assert test() == (2,2), “Y is wrong shape!”
print(“Correct!“)

#### Initializing the ResNet Model

In [ ]:
# ResNET Layers [3, 4, 6, 3] for 34 and 50
model = ResNET(Block, [3, 4, 6, 3], img_channel, num_classes)
model.to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
start_time = time.time()
minibatch_loss_list, train_acc_list, valid_acc_list = [], [], []

for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)
      
        features = torch.tensor(features, dtype=torch.float32)
        features = torch.transpose(features, 3, 1)#(x, 1, 3)
        targets = torch.tensor(targets, dtype=torch.float32)
      
        logits = model(features)
        cost = loss(logits, targets)
        optimizer.zero_grad()
        cost.backward()

        optimizer.step()
      
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                    %(epoch+1, num_epochs, batch_idx, 
                      len(train_loader), cost))
          
    model = model.eval() # eval mode to prevent upd. batchnorm params during inference
    with torch.set_grad_enabled(False):# save memory during inference
        train_acc, train_pred = calculate_accuracy(model, train_loader)
        valid_acc, valid_pred = calculate_accuracy(model, valid_loader)
        print('Epoch: %03d/%03d training accuracy: %.2f%% | Validation accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              train_acc, valid_acc))
        train_acc_list.append(train_acc.item())
        valid_acc_list.append(valid_acc.item())

    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

In [ ]:
# Saving the model for future use
#torch.save(model.state_dict(), 'resNET_model.pth')

#### Display the accuracy Plot:

In [ ]:
import matplotlib.pyplot as plt

num_epochs = len(train_acc_list)

plt.plot(np.arange(1, num_epochs+1),
          train_acc_list, label='Training')
plt.plot(np.arange(1, num_epochs+1),
          valid_acc_list, label='Validation')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()

#### Computing test accuracy and plotting a few examples with their predictions

In [ ]:
acc, predictions = calculate_accuracy(model, test_loader)
print('Test accuracy: %.2f%%' % (acc))

#%% Training data visualization
images, labels = next(iter(test_loader))

plt.figure(figsize=(5,5))
for i in range(9):
    
    plt.subplot(3, 3, i+1)
    label = np.argmax( labels[i] ,axis=0)
    
    ## Debugging
    #print(len(label))
    #print(predictions.shape)
    #print(images[i].shape, ' : ', images[i].permute(1, 2, 0).shape)
    #print(predictions[i].item(), '|', label.item())
    
    str = F"P: {predictions[i].item()},\nL: {label.item()}"
    plt.title(str)
    plt.imshow(images[i])
    
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
sns.set()

def get_predictions_and_labels(data_loader, device = 'cpu'):
    
    labels = None
    logits = None
    
    for batch_idx, (features, targets) in enumerate(data_loader):
        features = torch.tensor(features, dtype=torch.float32)
        features = torch.transpose(features, 3, 1)#(x, 1, 3)
        targets = torch.tensor(targets, dtype=torch.float32)
        
        features = features.to(device)
        targets = targets.to(device)
        
        logits = model(features)
        labels = targets
    
    labels = np.argmax( labels ,axis=1)
    logits = np.argmax( logits.detach().numpy() ,axis=1)
    
    return labels, logits

def get_classification_matrics(data_loader, target_names = ['0', '1'], device = 'cpu'):
    labels, prediction = get_predictions_and_labels(data_loader, device)
    
    print(classification_report(labels, prediction, target_names=target_names))
    
    print_confusion_matrix(labels, prediction, 
                           title = 'Confusion Matrix of ResNET',
                           x_lable = 'Predicted Label',
                           y_lable = 'True Label')
    
def print_confusion_matrix(labels, prediction, title, x_lable, y_lable, fig_size = 5):
    
    resNet_cm = confusion_matrix(labels, prediction)
    f,ax = plt.subplots(figsize=(fig_size, fig_size))
    sns.heatmap(resNet_cm, annot=True, linewidths=0.01,cmap="OrRd",linecolor="black", fmt= '.1f',ax=ax)
    plt.xlabel(y_lable)
    plt.ylabel(x_lable)
    plt.title(title)
    plt.show()
    

get_classification_matrics(test_loader)

## That's all folks!